    Instructions:
    1. Press Run All (or restart kernel and run all cells).
    2. You will be prompted to provide input values.


    Task 4:
    – 4a: Implement a Locality Sensitive Hashing (LSH) tool (for Euclidean distance) which takes as input (a) the number
    of layers, L, (b) the number of hashes per layer, h, and (c) a set of vectors as input and creates an in-memory index
    structure containing the given set of vectors. 
    
    See ”Near-Optimal Hashing Algorithms for Approximate Nearest Neighbor in High Dimensions” (by Alexandr
    Andoni and Piotr Indyk). Communications of the ACM, vol. 51, no. 1, 2008, pp. 117-122.

In [13]:
L = int(input("L - The number of layers"))

h = int(input("h - the number of hashes per layer"))

W = int(input("W - the number of buckets per hash"))

VECTORS_INPUT = input(
"""
A set of vectors. You have the following choices as input:
1. Provide the name of a JSON file in Code/input/ that contains vectors.
2. Provide a feature space, eg: resnet
"""
)

In [14]:
# Load corresponding vectors.

if VECTORS_INPUT.endswith(".json"):

    from utils.query_input_processor import get_vectors_from_file

    vectors = get_vectors_from_file(VECTORS_INPUT)
    img_ids = [None] * len(vectors)
else:

    FEATURE_SPACE = VECTORS_INPUT

    from utils.database_utils import retrieve
    feature_vectors = retrieve(f'{FEATURE_SPACE}.pt')

    from utils.vector_utils import feature_vectors_to_np_vectors

    vectors = feature_vectors_to_np_vectors(feature_vectors)
    img_ids = list(feature_vectors.keys())

In [15]:
print("Number of vectors: ", len(vectors))
print("Sample: ", vectors[0])

Number of vectors:  4339
Sample:  [ 5.30225933e-01  1.63977519e-01  1.99958198e-02 -2.56892703e-02
  2.13202968e-01 -3.04492831e-01 -4.58242208e-01  2.56279409e-01
 -3.45264375e-01 -9.38558429e-02  1.94023997e-01 -1.26296207e-01
 -3.79452229e-01 -1.81054398e-01 -7.08631501e-02 -4.71129715e-01
 -1.21493667e-01 -9.90327895e-02 -7.63641357e-01 -2.03926980e-01
  1.15600005e-02 -1.81398600e-01 -1.05172172e-02 -1.10233814e-01
  1.27047077e-01 -2.38607094e-01 -2.08716780e-01  1.34327590e-01
  1.53778091e-01 -1.30412191e-01 -1.87759131e-01 -1.31165430e-01
 -1.13427944e-01 -1.56426549e-01 -8.58325601e-01 -2.74171293e-01
  2.13534206e-01 -1.34047940e-01 -3.71960193e-01  1.00101143e-01
 -2.22137831e-02 -6.60765246e-02  3.85517389e-01 -1.36091709e-01
 -1.64662778e-01  1.29379928e-01 -2.65778512e-01  2.16090307e-03
 -2.16529854e-02 -2.86354512e-01 -3.21534760e-02 -4.12414372e-01
  4.10557956e-01  2.44809285e-01 -1.72773704e-01  2.77879760e-02
  2.32589126e-01 -2.67702073e-01 -6.53675944e-02  7.0575

In [16]:
# Index all vectors

from indexer.lsh import LSH

lsh = LSH(L=L, h=h, W=W, vectors=vectors)

for i, vector in enumerate(vectors):
    lsh.index(vector, img_ids[i])

In [17]:
# Test by querying a vector

IDX = 1250
res, unique_considered, total_considered = lsh.query(vectors[IDX], limit=10)

print("Locality-Sensitive Hashing Results for L = {}, h = {}, W = {}".format(L, h, W))
print("Image ID: ", img_ids[IDX], "\tUnique considered: ", unique_considered, "\tTotal considered: ", total_considered)

for i, vec_tup in enumerate(res):
    print(f"{i + 1}.\tImage ID: {vec_tup[1]}\tDistance: {vec_tup[2]}")

Locality-Sensitive Hashing Results for L = 10, h = 5, W = 5
Image ID:  2500 	Unique considered:  1231 	Total considered:  2032
1.	Image ID: 2500	Distance: 0.0
2.	Image ID: 2308	Distance: 4.294015407562256
3.	Image ID: 2670	Distance: 4.866341590881348
4.	Image ID: 2254	Distance: 4.904757976531982
5.	Image ID: 2252	Distance: 4.940627574920654
6.	Image ID: 2226	Distance: 5.0099263191223145
7.	Image ID: 2484	Distance: 5.320183753967285
8.	Image ID: 2134	Distance: 5.432594299316406
9.	Image ID: 2570	Distance: 5.615747928619385
10.	Image ID: 2492	Distance: 5.646214008331299


In [18]:
# Store LSH in memory structure to be used in later tasks.

from utils.database_utils import compressed_store

compressed_store(lsh, f"LSH_L{L}_h{h}_W{W}_{VECTORS_INPUT}.pt")


 Saving:  LSH_L10_h5_W5_resnet.pt 

